In [164]:
import pandas as pd
import numpy as np
import datetime

In [259]:
df6 = pd.read_csv("./data/6.범죄발생원표.csv")
df7 = pd.read_csv("./data/7.범죄검거원표.csv")
base = pd.read_csv("./data/base_df.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# 6. 범죄발생

In [260]:
# 경찰서명 공백제거
df6["jur_stn"] = df6["jur_stn"].str.replace(" ", "")

In [261]:
# 도명 제거
df6["jur_stn"] = df6["jur_stn"].str.replace("경기", "")
df6["jur_stn"] = df6["jur_stn"].str.replace("충남", "")
df6["jur_stn"] = df6["jur_stn"].str.replace("경남", "")

In [262]:
# base set과 data set이 다른 이름이 있는지 확인
df6_list = set(df6["jur_stn"].unique())
police_list = list(base["jur_stn"].unique())
plist_set = set(police_list)
df6_list - plist_set

set()

In [263]:
len(df6["jur_stn"].unique())

41

In [264]:
# 범죄발생날씨, 범죄단서정보, 피해자 연령, 범죄발생시간 column drop
df6.drop(["crm_wthr", "crm_clue", "vic_sx", "vic_age", "crm_tm"], axis=1, inplace=True)

In [265]:
# date 컬럼 type 변경
df6["crm_date"] = df6["crm_date"].astype("str")

In [266]:
# 연도, 월, 일 구분하여 datetime 적용
def _str_to_date(x) :
    try :
        x = datetime.datetime.strptime(x, '%Y%m%d')
    except ValueError :
        x = datetime.datetime.strptime(x, '%m/%d/%Y')
    return x 

df6['year'] = df6['crm_date'].map( lambda x : _str_to_date(x))

df6['year'] = pd.to_datetime(df6["year"])

In [267]:
# 상/하반기 나누는 함수
def _date_to_year(x) :
    if x < datetime.date(2017,1,1):
        return '2016(이전)'
    if x < datetime.date(2017, 7, 1) :
        return "2017(상반기)"
    if x < datetime.date(2018,1,1) :
        return "2017(하반기)"
    if x < datetime.date(2018,7,1) :
        return "2018(상반기)"
    if x < datetime.date(2019,1,1) :
        return "2018(하반기)"
    if x < datetime.date(2019,7,1) :
        return "2019(상반기)"
    return "2019(하반기)"

df6["year"] = df6["year"].map(lambda x : _date_to_year(x))

In [268]:
# base_df와 비교 했을때 새로운 2016(이전) 생성
set(df6["year"]) - set(base["date"])

{'2016(이전)'}

In [269]:
# 연도별 범죄 발생 건 수
df6["year"].value_counts()

2017(상반기)    199451
2017(하반기)    194428
2018(하반기)    185864
2018(상반기)    181755
2019(상반기)    174138
2019(하반기)    132604
2016(이전)       4982
Name: year, dtype: int64

In [271]:
# 기존 위치, date 컬럼 삭제
df6.drop(["crm_loc", "crm_date"], axis = 1, inplace = True)

In [272]:
df6

,jur_stn,crm,5m_crm_yn,year
0,서울수서경찰서,위조외국통화행사,,2017(상반기)
1,서울영등포경찰서,도로교통법위반,,2017(상반기)
2,서울양천경찰서,209015100,,2017(상반기)
3,서울서초경찰서,폭행,폭력,2017(상반기)
4,서울동대문경찰서,사기,,2017(상반기)
...,...,...,...,...
1073217,진해경찰서,사기,,2019(하반기)
1073218,진해경찰서,폭행,폭력,2019(하반기)
1073219,마산동부경찰서,재물손괴,폭력,2019(하반기)
1073220,마산중부경찰서,사기,,2019(하반기)


In [273]:
# 기존 공백 값 nan으로 변경 후 미분류로 변경
df6["5m_crm_yn"] = df6["5m_crm_yn"].replace(" ", np.NaN)
df6["5m_crm_yn"] = df6["5m_crm_yn"].fillna("미분류")

In [277]:
df6_tt = df6.groupby(["jur_stn", "year"], as_index=False).count()
df6_tt.drop("5m_crm_yn", axis = 1, inplace=True)

In [279]:
df6_tt.to_csv("./data/6번.범죄발생_proccessed.csv", index=False)

In [179]:
df6.loc[df6['crm'].str.contains('절도'), ['theft_violence']] = 1
df6.loc[df6['crm'].str.contains('폭력'), ['theft_violence']] = 1

df6.loc[df6['crm'].str.contains('강도'), ['robber_murder']] = 1
df6.loc[df6['crm'].str.contains('살인'), ['robber_murder']] = 1

df6.loc[df6['crm'].str.contains('교통'), ['traffic']] = 1

df6.loc[df6['crm'].str.contains('집회'), ['public']] = 1
df6.loc[df6['crm'].str.contains('광고물'), ['public']] = 1
df6.loc[df6['crm'].str.contains('국민기초'), ['public']] = 1

In [180]:
df6_test = df6.groupby(["jur_stn", "year"], as_index=False).count()

In [182]:
df6_test.to_csv("./data/6번.범죄발생_total.csv", index=False)